In [2]:
#!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [6]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [7]:
os.environ["GROQ_API_KEY"] = "your_groq_api_key"

In [8]:
# laoding the document
loader = UnstructuredFileLoader("attention_is_all_you_need.pdf")
documents = loader.load()

/home/sid/jupyter/myenv/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [10]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [11]:
texts = text_splitter.split_documents(documents)

In [12]:
type(texts)

list

In [13]:
len(texts)

26

In [15]:
#texts[:3]

In [16]:
embeddings = HuggingFaceEmbeddings()

/home/sid/jupyter/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [18]:
persist_directory = "doc_db"

In [19]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [20]:
# retriever
retriever = vectordb.as_retriever()

In [21]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [22]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [23]:
# invoke the qa chain and get a response for user query
query = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})

In [24]:
print(response)

{'query': 'What is the model architecture discussed in this paper?', 'result': 'The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs multi-head attention over the output of the encoder stack. The model uses residual connections and layer normalization.', 'source_documents': [Document(metadata={'source': 'attention_is_all_you_need.pdf'}, page_content='2\n\nFigure 1: The Transformer - model architecture.\n\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.\n\n3.1 Encoder and Decoder Stacks\n\nEncoder: The encoder is composed of a 

In [25]:
print(response["result"])

The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs multi-head attention over the output of the encoder stack. The model uses residual connections and layer normalization.


In [26]:
print(response["source_documents"][0].metadata["source"])

attention_is_all_you_need.pdf


In [29]:
# invoke the qa chain and get a response for user query
query = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs multi-head attention over the output of the encoder stack. The model uses residual connections and layer normalization to facilitate training.
******************************
Source Document: attention_is_all_you_need.pdf
